## Flight refund process

In [1]:

from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key
)

def refund_flight(flight_id: str) -> str:
    """Refund a flight"""
    return f"Flight {flight_id} refunded"

In [2]:
model_client = azure_openai_chat_completion_client

travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)


In [3]:
task = "I need to refund my flight."


async def run_team_stream() -> None:
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]

    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        user_message = input("User: ")

        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]


await run_team_stream()

---------- user ----------
I need to refund my flight.


---------- travel_agent ----------
[FunctionCall(id='call_CMxZvweIPrMSOP4JgS0n6E7C', arguments='{}', name='transfer_to_flights_refunder')]
[Prompt tokens: 119, Completion tokens: 35]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', call_id='call_CMxZvweIPrMSOP4JgS0n6E7C')]
---------- travel_agent ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.


/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:343: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


---------- flights_refunder ----------
I can help with that. May I please have your flight reference number?
[Prompt tokens: 191, Completion tokens: 16]
---------- flights_refunder ----------
[FunctionCall(id='call_FHtCOys1ZLVm9dRkcfujQmdY', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 215, Completion tokens: 11]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', call_id='call_FHtCOys1ZLVm9dRkcfujQmdY')]
---------- flights_refunder ----------
Transferred to user, adopting the role of user immediately.
---------- Summary ----------
Number of messages: 8
Finish reason: Handoff to user from flights_refunder detected.
Total prompt tokens: 525
Total completion tokens: 62
Duration: 2.47 seconds
---------- user ----------
A11
---------- flights_refunder ----------
[FunctionCall(id='call_tJ8omk03YGi7EKhwEsnx08wk', arguments='{"flight_id":"A11"}', name='refund_flight')]
[Prompt tokens: 258, Completi

## Stock Research

In [4]:
async def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Get stock market data for a given symbol"""
    return {"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"}


async def get_news(query: str) -> List[Dict[str, str]]:
    """Get recent news articles about a company"""
    return [
        {
            "title": "Tesla Expands Cybertruck Production",
            "date": "2024-03-20",
            "summary": "Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.",
        },
        {
            "title": "Tesla FSD Beta Shows Promise",
            "date": "2024-03-19",
            "summary": "Latest Full Self-Driving beta demonstrates significant improvements in urban navigation and safety features.",
        },
        {
            "title": "Model Y Dominates Global EV Sales",
            "date": "2024-03-18",
            "summary": "Tesla's Model Y becomes best-selling electric vehicle worldwide, capturing significant market share.",
        },
    ]

In [5]:
model_client = azure_openai_chat_completion_client
planner = AssistantAgent(
    "planner",
    model_client=model_client,
    handoffs=["financial_analyst", "news_analyst", "writer"],
    system_message="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
)

financial_analyst = AssistantAgent(
    "financial_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_stock_data],
    system_message="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    Always handoff back to planner when analysis is complete.""",
)

news_analyst = AssistantAgent(
    "news_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_news],
    system_message="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
)

writer = AssistantAgent(
    "writer",
    model_client=model_client,
    handoffs=["planner"],
    system_message="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    Always handoff back to planner when writing is complete.""",
)

In [10]:
# Define termination condition
text_termination = TextMentionTermination("TERMINATE")
termination = text_termination

research_team = Swarm(
    participants=[planner, financial_analyst, news_analyst, writer], termination_condition=termination
)

task = "Conduct market research for TSLA stock"
response = await Console(research_team.run_stream(task=task))

---------- user ----------
Conduct market research for TSLA stock


---------- planner ----------
Plan:
1. Financial analysis for TSLA stock data.
2. News gathering and analysis for TSLA.
3. Compilation of the final report by the writer.

Let's start with the financial analysis.
[Prompt tokens: 557, Completion tokens: 41]


/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:307: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


---------- planner ----------
[FunctionCall(id='call_Ho9UGiafLLncARrJ96bdKF2i', arguments='{}', name='transfer_to_financial_analyst')]
[Prompt tokens: 603, Completion tokens: 15]
---------- planner ----------
[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', call_id='call_Ho9UGiafLLncARrJ96bdKF2i')]
---------- planner ----------
Transferred to financial_analyst, adopting the role of financial_analyst immediately.
---------- financial_analyst ----------
Here are the insights from the recent TSLA stock data:

1. **Price**: The stock price is $180.25.
2. **Volume**: The trading volume is 1,000,000 shares.
3. **P/E Ratio**: The price-to-earnings (P/E) ratio is 65.4.
4. **Market Capitalization**: The market cap is $700 billion.

### Analysis:
- **Stock Price**: A stock price of $180.25 indicates where the market currently values TSLA. This price can be affected by a variety of factors, including market sentiment, compan

In [17]:
print(response.messages[-7].content)

### TSLA Stock Market Research Report

**Stock Performance Overview:**

1. **Stock Price**: $180.25
2. **Trading Volume**: 1,000,000 shares
3. **P/E Ratio**: 65.4
4. **Market Capitalization**: $700 billion

**Financial Analysis:**

The current stock price of $180.25 reflects market valuation based on several influencing factors such as company performance and broader economic conditions. The trading volume of 1,000,000 shares indicates high investor interest and potential for price movements. A P/E ratio of 65.4 shows that investors are expecting significant future growth, although it might also suggest the stock is overvalued relative to its earnings. With a market cap of $700 billion, Tesla stands as one of the largest companies by market capitalization, highlighting its strong market position and investor confidence.

**Recent News and Impact:**

1. **Tesla Expands Cybertruck Production**: 
   Tesla is scaling up the production capacity for its Cybertruck at Gigafactory Texas in res